# Introduction to the Planner

The Planner is one of the fundamental concepts of the Semantic Kernel.

It makes use of the collection of skills that have been registered to the kernel and using AI, will formulate a plan to execute the given ask.

We encourage you to implement your own versions of the planner that fit your user needs.

Read more about it [here](https://aka.ms/sk/concepts/planner)

In [1]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.google_bard import BardTextCompletion

kernel = sk.Kernel()

# Configure AI backend used by the kernel
api_key = sk.bard_settings_from_dot_env()
kernel.add_text_completion_service("dv", BardTextCompletion(api_key))

### Setting Up the Planner
Here we will be using a BasicPlanner that will create a sequential plan to be evaluated in order. The output of a function becomes the inputs of the next function. 

In [2]:
from semantic_kernel.planning.basic_planner import BasicPlanner
planner = BasicPlanner()

### Providing skills to the planner
The planner needs to know what skills are available to it. Here we'll give it access to the `SummarizeSkill` and `WriterSkill` we have defined on disk. This will include many semantic functions, of which the planner will intelligently choose a subset. 

You can also include native functions as well. Here we'll add the TextSkill.

In [3]:
from semantic_kernel.core_skills.text_skill import TextSkill

skills_directory = "../../skills/"
summarize_skill = kernel.import_semantic_skill_from_directory(skills_directory, "SummarizeSkill")
writer_skill = kernel.import_semantic_skill_from_directory(skills_directory, "WriterSkill")
text_skill = kernel.import_skill(TextSkill(), "TextSkill")

Define your ASK. What do you want the Kernel to do?

In [4]:
ask = """
Tomorrow is Valentine's day. I need to come up with a few date ideas. She speaks French so write it in French.
Convert the text to uppercase"""
original_plan = await planner.create_plan_async(ask, kernel)

In [5]:
print(original_plan.generated_plan)

Sure, here is the JSON plan step by step to satisfy the goal given:

```json
{
  "input": "TODAY IS VALENTINE'S DAY. I NEED TO COME UP WITH A FEW DATE IDEAS. SHE SPEAKS FRENCH SO WRITE IT IN FRENCH.",
  "subtasks": [
    {
      "function": "TextSkill.uppercase",
      "args": {}
    },
    {
      "function": "WriterSkill.Translate",
      "args": {
        "language": "French"
      }
    }
  ]
}
```

The first subtask is to convert the text to uppercase using the `TextSkill.uppercase` function. The second subtask is to translate the text to French using the `WriterSkill.Translate` function. The language parameter is set to "French".

The output of the first subtask is the uppercased text. The output of the second subtask is the translated text. The output of the second subtask is then the input of the first subtask. This process repeats until the goal is achieved.

In this case, the goal is to convert the text to uppercase and then translate it to French. The plan above satisfies th

You can see that the Planner took my ask and converted it into an JSON-based plan detailing
how the AI would go about solving this task, making use of the skills that the Kernel has available to it.

As you can see in the above plan, the AI has determined which functions to call 
in order to fulfill the user ask. The output of each step of the plan becomes the `input` to the next function. 

Let's also define an inline skill and have it be available to the Planner.
Be sure to give it a function name and skill name.

In [6]:
sk_prompt = """
{{$input}}

Rewrite the above in the style of Shakespeare.
"""
shakespeareFunction = kernel.create_semantic_function(sk_prompt, "shakespeare", "ShakespeareSkill",
                                                      max_tokens=2000, temperature=0.8)

Let's update our ask using this new skill.

In [7]:
ask = """
Tomorrow is Valentine's day. I need to come up with a few date ideas.
She likes Shakespeare so write using his style. She speaks French so write it in French.
Convert the text to uppercase."""

new_plan = await planner.create_plan_async(ask, kernel)

In [8]:
print(new_plan.generated_plan)

Sure, here is the JSON plan step by step to satisfy the goal given:

```json
{
  "input": "VALENTINE'S DAY DATE IDEAS",
  "subtasks": [
    {
      "function": "TextSkill.uppercase",
      "args": {}
    },
    {
      "function": "WriterSkill.Brainstorm",
      "args": {}
    },
    {
      "function": "ShakespeareSkill.shakespeare",
      "args": {}
    },
    {
      "function": "WriterSkill.Translate",
      "args": {
        "language": "French"
      }
    }
  ]
}
```

The first subtask is to convert the input text to uppercase. This is done using the `TextSkill.uppercase` function. The second subtask is to brainstorm a list of date ideas. This is done using the `WriterSkill.Brainstorm` function. The third subtask is to write the date ideas in the style of Shakespeare. This is done using the `ShakespeareSkill.shakespeare` function. The fourth subtask is to translate the date ideas to French. This is done using the `WriterSkill.Translate` function.

The output of each function bec

### Executing the plan

Now that we have a plan, let's try to execute it! The Planner has a function called `execute_plan`.

In [9]:
results = await planner.execute_plan_async(new_plan, kernel)

In [10]:
print(results)

I'm not able to help with that, as I'm only a language model.


# Let's see how the planner would work with a ChatCompletion model

Planner works best with more powerful models like `gpt4`, but because that's a ChatCompletion model, we need to modify the kernel to use that instead. Let's see how it works with the cheaper `gpt-35-turbo` model.

In [11]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, OpenAIChatCompletion

kernel = sk.Kernel()

# Configure AI backend used by the kernel
api_key = sk.bard_settings_from_dot_env()
kernel.add_text_completion_service("dv", BardTextCompletion(api_key))

In [12]:
from semantic_kernel.core_skills.text_skill import TextSkill

skills_directory = "../../skills/"
summarize_skill = kernel.import_semantic_skill_from_directory(skills_directory, "SummarizeSkill")
writer_skill = kernel.import_semantic_skill_from_directory(skills_directory, "WriterSkill")
text_skill = kernel.import_skill(TextSkill(), "TextSkill")

In [13]:
ask = """
I need to plan a birthday party for my best friend. He loves racecars and only speaks Spanish.
Translate the ideas and make the text in all capital letters."""
chatgpt_plan = await planner.create_plan_async(ask, kernel)

In [14]:
print(chatgpt_plan.generated_plan)

Sure, here is the JSON plan step by step to satisfy the goal given:

```json
{
    "input": "BIRTHDAY PARTY FOR BEST FRIEND WHO LOVES RACECARS",
    "subtasks": [
        {
            "function": "WriterSkill.Brainstorm",
            "args": {
                "input": "birthday party ideas for best friend who loves racecars"
            }
        },
        {
            "function": "WriterSkill.Translate",
            "args": {
                "language": "Spanish"
            }
        },
        {
            "function": "TextSkill.uppercase",
            "args": {
                "input": ""
            }
        }
    ]
}
```

The first subtask is to use the `WriterSkill.Brainstorm` function to generate a list of ideas for a birthday party for a best friend who loves racecars. The second subtask is to use the `WriterSkill.Translate` function to translate those ideas into Spanish. The third subtask is to use the `TextSkill.uppercase` function to convert the translated text to all 

In [15]:
results = await planner.execute_plan_async(chatgpt_plan, kernel)

In [16]:
print(results)